In [2]:
import sqlite3
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
# Load the model
model = load_model('updated_electricity_price_model.h5')

2024-06-27 18:35:24.133387: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-06-27 18:35:24.133444: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: ashagaire-Lenovo-Legion-5-15ARH05
2024-06-27 18:35:24.133465: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: ashagaire-Lenovo-Legion-5-15ARH05
2024-06-27 18:35:24.133651: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-06-27 18:35:24.133712: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.183.1
2024-06-27 18:35:24.133730: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 535.183.1


In [4]:
# Connect to the SQLite database
conn = sqlite3.connect("../prisma/db.sqlite")
# Query recent new feature data
query = "SELECT * FROM HistoricalElectricityWeather WHERE price IS NULL "
data = pd.read_sql_query(query, conn)
data.shape

(314, 11)

In [5]:
data.head()

,id,dateId,price,temperature,precipitation,cloudCover,windSpeed10m,shortwaveRadiation,weatherCodeId,createdDate,modifiedDate
0,2045,2045,None,-10.8850,0.0,19.500002,10.195057,0,0,2024-06-13 10:55:45,2024-06-13 10:55:45
1,10781,10781,None,-15.8350,0.0,0.000000,7.244860,0,0,2024-06-13 10:55:45,2024-06-13 10:55:45
2,19685,19685,None,-0.7350,0.0,92.699997,6.409617,1,3,2024-06-13 10:55:45,2024-06-13 10:55:45
3,21482,21482,None,7.7475,0.0,99.000000,1.800000,8,3,2024-06-13 10:55:45,2024-06-13 10:55:45
4,21483,21483,None,8.0475,0.0,97.000000,1.440000,41,3,2024-06-13 10:55:45,2024-06-13 10:55:45


In [6]:
# Select specific columns using .filter
X = data.filter(['temperature', 'precipitation', 'weatherCodeId', 'cloudCover', 'windSpeed10m', 'shortwaveRadiation'])

In [7]:

# Load the scalers from disk
scaler_X = joblib.load('scaler_X.pkl')
scaler_y = joblib.load('scaler_y.pkl')

/home/ashagaire/Desktop/LearnIT and coding with Finnish Language/Mittwatt/electricity-shoc/mittwatt/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
# Normalize the features
X_normalized = scaler_X.transform(X)
# Predict using the trained model
y_pred_scaled = model.predict(X_normalized)
# Inverse transform the predicted values to get them back to the original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_pred_flatten = y_pred.flatten()
y_pred_one_d = y_pred_flatten.astype(float).round(2)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [39]:
# Create the new DataFrame
prediction_dataframe_new = pd.DataFrame({
    'dateId': data['dateId'],
    'price': y_pred_one_d
})

In [41]:
# Insert or update rows
for index, row in prediction_dataframe_new.iterrows():
    date = row['dateId']
    price = row['price']
    insert_or_replace_query = """
    INSERT OR REPLACE INTO ForecastElectricityPrice (dateId, price)
    VALUES (?, ?);
    """
    conn.execute(insert_or_replace_query, (date, price))


In [51]:
# Close the connection
conn.commit()
conn.close()